# Formation de Structures dans le Modèle ΛCDM
## Prédictions pour les Galaxies Primordiales (z > 8)

**Projet**: VAL-Galaxies_primordiales - Phase 1.1.2

**Date**: 6 Janvier 2026

**Objectif**: Calculs détaillés des prédictions théoriques du modèle ΛCDM standard pour la formation des galaxies à haut redshift, servant de référence pour comparaison avec JANUS.

## 1. Configuration et Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
import sys
from pathlib import Path

# Ajouter src au path
src_path = Path('../../src')
sys.path.insert(0, str(src_path))

# Import des modules cosmologiques
from cosmology import LCDMCosmology, JANUSCosmology

# Configuration matplotlib
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Imports réussis!")

## 2. Initialisation du Modèle ΛCDM (Planck 2018)

In [ ]:
# Paramètres Planck 2018
lcdm = LCDMCosmology(
    H0=67.4,           # Constante de Hubble [km/s/Mpc]
    Omega_m=0.315,     # Densité matière totale
    Omega_Lambda=0.685 # Densité énergie sombre
)

print(lcdm)
print(f"\nParamètres Planck 2018:")
print(f"  H₀ = {lcdm.H0} km/s/Mpc")
print(f"  Ωₘ = {lcdm.Omega_m}")
print(f"  ΩΛ = {lcdm.Omega_Lambda}")
print(f"  Ωₘ + ΩΛ = {lcdm.Omega_m + lcdm.Omega_Lambda:.3f} (univers plat)")

## 3. Paramètre de Hubble H(z)

In [ ]:
# Calcul de H(z) sur une grille de redshifts
z_grid = np.linspace(0, 20, 200)
H_lcdm = lcdm.hubble_parameter(z_grid)

# Visualisation
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(z_grid, H_lcdm, 'r-', linewidth=2, label='ΛCDM (Planck 2018)')
ax.axhline(y=lcdm.H0, color='gray', linestyle='--', label=f'H₀ = {lcdm.H0} km/s/Mpc')

ax.set_xlabel('Redshift z')
ax.set_ylabel('H(z) [km/s/Mpc]')
ax.set_title('Paramètre de Hubble - Modèle ΛCDM')
ax.legend()
ax.set_xlim(0, 20)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Vérification H(z=0)
print(f"\nH(z=0) = {lcdm.hubble_parameter(0):.2f} km/s/Mpc")
print(f"H(z=10) = {lcdm.hubble_parameter(10):.2f} km/s/Mpc")

## 4. Âge de l'Univers - Comparaison ΛCDM vs JANUS

In [ ]:
# Initialisation JANUS pour comparaison
janus = JANUSCosmology(H0=70.0, Omega_plus=0.30, Omega_minus=0.05)

# Redshifts d'intérêt pour les galaxies primordiales
z_highz = np.array([0, 2, 4, 6, 8, 10, 12, 14, 15])

# Calcul des âges
ages_lcdm = np.array([lcdm.age_of_universe(z) for z in z_highz])
ages_janus = np.array([janus.age_of_universe(z) for z in z_highz])

# Tableau des résultats
print("Comparaison des Âges de l'Univers")
print("=" * 65)
print(f"{'Redshift':>10} | {'ΛCDM (Gyr)':>12} | {'JANUS (Gyr)':>12} | {'Δt (Myr)':>12}")
print("-" * 65)
for z, age_l, age_j in zip(z_highz, ages_lcdm, ages_janus):
    delta = (age_j - age_l) * 1000  # en Myr
    print(f"{z:>10.1f} | {age_l:>12.3f} | {age_j:>12.3f} | {delta:>+12.0f}")

In [ ]:
# Visualisation comparative
z_fine = np.linspace(0, 15, 100)
ages_lcdm_fine = np.array([lcdm.age_of_universe(z) for z in z_fine])
ages_janus_fine = np.array([janus.age_of_universe(z) for z in z_fine])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Panel 1: Âges absolus
ax1.plot(z_fine, ages_lcdm_fine, 'r-', linewidth=2, label='ΛCDM')
ax1.plot(z_fine, ages_janus_fine, 'b-', linewidth=2, label='JANUS')
ax1.scatter(z_highz[4:8], ages_lcdm[4:8], color='red', s=100, zorder=5)
ax1.scatter(z_highz[4:8], ages_janus[4:8], color='blue', s=100, zorder=5)

ax1.set_xlabel('Redshift z')
ax1.set_ylabel('Âge de l\'Univers [Gyr]')
ax1.set_title('Évolution de l\'Âge')
ax1.legend()
ax1.set_xlim(0, 16)
ax1.grid(True, alpha=0.3)

# Panel 2: Différence relative
delta_percent = (ages_janus_fine - ages_lcdm_fine) / ages_lcdm_fine * 100
ax2.plot(z_fine, delta_percent, 'g-', linewidth=2)
ax2.axhline(y=0, color='gray', linestyle='--')
ax2.fill_between(z_fine, 0, delta_percent, alpha=0.3, color='green')

ax2.set_xlabel('Redshift z')
ax2.set_ylabel('(JANUS - ΛCDM) / ΛCDM [%]')
ax2.set_title('Différence Relative des Âges')
ax2.set_xlim(0, 16)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Distances Cosmologiques

In [ ]:
# Calcul des distances ΛCDM
z_dist = np.linspace(0.1, 15, 100)

d_comoving_lcdm = np.array([lcdm.comoving_distance(z) for z in z_dist])
d_luminosity_lcdm = np.array([lcdm.luminosity_distance(z) for z in z_dist])
d_angular_lcdm = np.array([lcdm.angular_diameter_distance(z) for z in z_dist])

# Visualisation
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(z_dist, d_comoving_lcdm/1000, 'b-', linewidth=2, label='Distance comobile')
ax.plot(z_dist, d_luminosity_lcdm/1000, 'r-', linewidth=2, label='Distance lumineuse')
ax.plot(z_dist, d_angular_lcdm/1000, 'g-', linewidth=2, label='Distance angulaire')

ax.set_xlabel('Redshift z')
ax.set_ylabel('Distance [Gpc]')
ax.set_title('Distances Cosmologiques - Modèle ΛCDM')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Contrainte Temporelle: Le "Problème des Galaxies Impossibles"

In [ ]:
# Temps disponible pour la formation stellaire en ΛCDM
z_obs = np.array([8, 10, 12, 14])
t_lcdm = np.array([lcdm.age_of_universe(z) for z in z_obs]) * 1000  # Myr
t_janus = np.array([janus.age_of_universe(z) for z in z_obs]) * 1000  # Myr

print("Contrainte Temporelle pour Formation Stellaire")
print("=" * 80)
print(f"{'z_obs':>6} | {'t_ΛCDM (Myr)':>12} | {'t_JANUS (Myr)':>13} | {'SFR_min ΛCDM':>14} | {'SFR_min JANUS':>14}")
print("-" * 80)

# Pour former M* = 10^10 M_sun
M_star_target = 1e10  # M_sun

for z, t_l, t_j in zip(z_obs, t_lcdm, t_janus):
    SFR_min_lcdm = M_star_target / (t_l * 1e6)  # M_sun/yr
    SFR_min_janus = M_star_target / (t_j * 1e6)  # M_sun/yr
    print(f"{z:>6.0f} | {t_l:>12.0f} | {t_j:>13.0f} | {SFR_min_lcdm:>11.0f} M☉/yr | {SFR_min_janus:>11.0f} M☉/yr")

print("\n→ Pour M* = 10¹⁰ M☉, ΛCDM requiert des SFR plus élevés (galaxies 'impossibles')")

## 7. Prédictions de Masse Stellaire Maximale

In [ ]:
def max_stellar_mass(z, sfr, cosmo):
    """
    Calcule la masse stellaire maximale possible à redshift z
    avec un SFR constant depuis le Big Bang.
    """
    age_gyr = cosmo.age_of_universe(z)
    age_yr = age_gyr * 1e9
    return sfr * age_yr

# Grille de SFR
sfr_values = [10, 50, 100, 200, 500]  # M_sun/yr
z_grid_mass = np.array([8, 10, 12, 14])

print("Masse Stellaire Maximale [10¹⁰ M☉]")
print("=" * 90)
print("\nModèle ΛCDM:")
header = f"{'z':>4} |" + " | ".join([f'SFR={sfr}' for sfr in sfr_values])
print(header)
print("-" * 60)

for z in z_grid_mass:
    masses = [max_stellar_mass(z, sfr, lcdm) / 1e10 for sfr in sfr_values]
    row = f"{z:>4.0f} |" + " | ".join([f'{m:>8.2f}' for m in masses])
    print(row)

print("\nModèle JANUS:")
print(header)
print("-" * 60)

for z in z_grid_mass:
    masses = [max_stellar_mass(z, sfr, janus) / 1e10 for sfr in sfr_values]
    row = f"{z:>4.0f} |" + " | ".join([f'{m:>8.2f}' for m in masses])
    print(row)

## 8. Comparaison avec Observations JWST

In [ ]:
# Données observationnelles JWST
obs_data = {
    'Labbé+23 #1': {'z': 7.4, 'log_M': 10.9, 'err': 0.3},
    'Labbé+23 #2': {'z': 9.1, 'log_M': 10.6, 'err': 0.3},
    'AC-2168': {'z': 12.15, 'log_M': 10.0, 'err': 0.5},
}

# Prédictions avec SFR=100 M_sun/yr
z_pred = np.linspace(6, 15, 50)
M_max_lcdm = np.array([max_stellar_mass(z, 100, lcdm) for z in z_pred])
M_max_janus = np.array([max_stellar_mass(z, 100, janus) for z in z_pred])

# Visualisation
fig, ax = plt.subplots(figsize=(12, 8))

# Régions de prédiction
ax.fill_between(z_pred, np.log10(M_max_lcdm*0.5), np.log10(M_max_lcdm*1.5),
                alpha=0.2, color='red', label='ΛCDM (SFR=50-150 M☉/yr)')
ax.fill_between(z_pred, np.log10(M_max_janus*0.5), np.log10(M_max_janus*1.5),
                alpha=0.2, color='blue', label='JANUS (SFR=50-150 M☉/yr)')

# Lignes centrales
ax.plot(z_pred, np.log10(M_max_lcdm), 'r-', linewidth=2, label='ΛCDM (SFR=100 M☉/yr)')
ax.plot(z_pred, np.log10(M_max_janus), 'b-', linewidth=2, label='JANUS (SFR=100 M☉/yr)')

# Observations
colors = ['orange', 'green', 'purple']
for (name, data), c in zip(obs_data.items(), colors):
    ax.errorbar(data['z'], data['log_M'], yerr=data['err'],
                fmt='s', markersize=12, capsize=5, color=c, label=name,
                markeredgecolor='black', markeredgewidth=1)

ax.set_xlabel('Redshift z', fontsize=14)
ax.set_ylabel('log(M★/M☉)', fontsize=14)
ax.set_title('Masse Stellaire Maximale: ΛCDM vs JANUS vs Observations JWST', fontsize=14)
ax.legend(loc='upper right', fontsize=10)
ax.set_xlim(6, 15)
ax.set_ylim(9, 12)
ax.grid(True, alpha=0.3)

# Zone de tension
ax.axhspan(10.5, 12, xmin=0.6, xmax=1, alpha=0.1, color='red', label='_nolegend_')
ax.text(13.5, 11.5, 'Zone de\ntension ΛCDM', ha='center', fontsize=10, color='red')

plt.tight_layout()
plt.show()

## 9. Analyse Quantitative des Tensions

In [ ]:
# Calcul des tensions pour chaque observation
print("Analyse des Tensions ΛCDM")
print("=" * 80)
print(f"{'Objet':>15} | {'z':>5} | {'M* obs':>10} | {'M* max ΛCDM':>12} | {'Tension':>10}")
print("-" * 80)

for name, data in obs_data.items():
    z = data['z']
    log_M_obs = data['log_M']
    M_obs = 10**log_M_obs
    
    # Masse max ΛCDM avec SFR=100 M_sun/yr
    M_max_l = max_stellar_mass(z, 100, lcdm)
    M_max_j = max_stellar_mass(z, 100, janus)
    
    # Ratio = M_obs / M_max
    ratio_lcdm = M_obs / M_max_l
    
    if ratio_lcdm > 1:
        tension = f"{ratio_lcdm:.1f}× excès"
    else:
        tension = "Compatible"
    
    print(f"{name:>15} | {z:>5.2f} | {log_M_obs:>10.1f} | {np.log10(M_max_l):>12.2f} | {tension:>10}")

print("\n→ Plusieurs galaxies JWST dépassent les limites ΛCDM théoriques")

## 10. Volume Comobile et Densité de Halos

In [ ]:
# Volume comobile à différents redshifts
z_vol = np.array([6, 8, 10, 12, 14])
V_c_lcdm = np.array([lcdm.comoving_volume(z) for z in z_vol])
V_c_janus = np.array([janus.comoving_volume(z) for z in z_vol])

print("Volume Comobile - Comparaison")
print("=" * 70)
print(f"{'Redshift':>10} | {'V_c ΛCDM (Gpc³)':>18} | {'V_c JANUS (Gpc³)':>18} | {'Ratio':>8}")
print("-" * 70)
for z, v_l, v_j in zip(z_vol, V_c_lcdm, V_c_janus):
    ratio = v_j / v_l
    print(f"{z:>10.0f} | {v_l/1e9:>18.2f} | {v_j/1e9:>18.2f} | {ratio:>8.2f}")

## 11. Fonction de Luminosité UV Théorique

In [ ]:
def schechter_function(M_UV, M_star=-20.5, phi_star=1e-4, alpha=-2.0):
    """
    Fonction de Schechter pour la fonction de luminosité UV.
    
    Parameters
    ----------
    M_UV : array
        Magnitude UV absolue
    M_star : float
        Magnitude caractéristique
    phi_star : float
        Normalisation [Mpc^-3 mag^-1]
    alpha : float
        Pente faint-end
    
    Returns
    -------
    phi : array
        Densité numérique [Mpc^-3 mag^-1]
    """
    x = 10**(0.4 * (M_star - M_UV))
    return 0.4 * np.log(10) * phi_star * x**(alpha + 1) * np.exp(-x)

# Paramètres à différents redshifts (Bouwens+21)
params_z = {
    8: {'M_star': -20.3, 'phi_star': 3e-4, 'alpha': -1.9},
    10: {'M_star': -20.5, 'phi_star': 1e-4, 'alpha': -2.0},
    12: {'M_star': -20.7, 'phi_star': 3e-5, 'alpha': -2.1},
}

M_UV = np.linspace(-24, -17, 100)

fig, ax = plt.subplots(figsize=(10, 7))

colors = ['blue', 'green', 'red']
for (z, params), c in zip(params_z.items(), colors):
    phi = schechter_function(M_UV, **params)
    ax.semilogy(M_UV, phi, '-', color=c, linewidth=2, label=f'z = {z}')

ax.set_xlabel('M_UV [mag]')
ax.set_ylabel('φ(M_UV) [Mpc⁻³ mag⁻¹]')
ax.set_title('Fonction de Luminosité UV - Prédictions ΛCDM')
ax.legend()
ax.set_xlim(-24, -17)
ax.set_ylim(1e-8, 1e-2)
ax.invert_xaxis()
ax.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

## 12. Résumé et Conclusions

In [ ]:
print("="*80)
print("RÉSUMÉ - Prédictions ΛCDM pour Galaxies Primordiales")
print("="*80)
print()
print("PARAMÈTRES PLANCK 2018:")
print(f"  H₀ = {lcdm.H0} km/s/Mpc")
print(f"  Ωₘ = {lcdm.Omega_m}")
print(f"  ΩΛ = {lcdm.Omega_Lambda}")
print()
print("PRÉDICTIONS CLÉS ΛCDM:")
print(f"  Âge à z=10: {lcdm.age_of_universe(10)*1000:.0f} Myr")
print(f"  Âge à z=12: {lcdm.age_of_universe(12)*1000:.0f} Myr")
print(f"  Âge à z=14: {lcdm.age_of_universe(14)*1000:.0f} Myr")
print()
print("COMPARAISON AVEC JANUS:")
print(f"  Δt(z=10): +{(janus.age_of_universe(10) - lcdm.age_of_universe(10))*1000:.0f} Myr")
print(f"  Δt(z=12): +{(janus.age_of_universe(12) - lcdm.age_of_universe(12))*1000:.0f} Myr")
print(f"  Δt(z=14): +{(janus.age_of_universe(14) - lcdm.age_of_universe(14))*1000:.0f} Myr")
print()
print("TENSIONS OBSERVATIONNELLES:")
print("  - Galaxies JWST à z > 10 avec M* > prédictions ΛCDM")
print("  - Abondance de galaxies brillantes 10-100× supérieure aux attentes")
print("  - Maturité morphologique et chimique précoce inattendue")
print()
print("CONCLUSION:")
print("  Le modèle ΛCDM fait face à des tensions significatives avec")
print("  les observations JWST de galaxies primordiales.")
print("  Le modèle JANUS, avec son chronologie plus longue, pourrait")
print("  résoudre ces tensions naturellement.")
print("="*80)

---

**Notebook créé le**: 6 Janvier 2026

**Version**: 1.0

**Référence**: PLAN.md Phase 1.1.2